In [1]:
import sys
import os
from pathlib import Path
import torch
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [2]:
args = parser.parse_args([])
args.DEVICE = 'cpu'
args.task = 'temporal'
args.metric = 'temporal_acc'
args.SoftEva = True
args = FormulateArgs(args)
args.N_test = 10
args.e_test = 0.1

In [3]:
args.augment = False
args.augment


False

In [4]:
results = torch.zeros([15])

for ds in range(15):
        args.DATASET = ds
        seed = args.SEED
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
        print(datainfo)

        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')
        # aug_model_exist = os.path.isfile(f'../AugLearnableFilters/models/{modelname}')
        # baseline_model_exist = os.path.isfile(f'../LastBaseline/LearnableFilters/models/{modelname}')
        
        # print(model_exist and aug_model_exist and baseline_model_exist)
        # print(model_exist and aug_model_exist)
        
        
        # if model_exist and aug_model_exist:
        if model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # aug_model = torch.load(f'../AugLearnableFilters/models/{modelname}', map_location=args.DEVICE)
            # baseline_model = torch.load(f'../LastBaseline/LearnableFilters/models/{modelname}', map_location=baseline_args.DEVICE)
            
            model.UpdateArgs(args)
            # aug_model.UpdateArgs(args)
            # baseline_model.UpdateArgs(baseline_args)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)
                start_time = time.time()
                model(X_test[:1, :, :])
                end_time = time.time()
                inference_time = end_time - start_time
                results[ds] = inference_time
                break

        else:
            results[ds] = float('nan')

{'dataname': 'cbf', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'distalphalanxtw', 'N_feature': 1, 'N_class': 6, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'freezerregulartrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'freezersmalltrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'gunpointagespan', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'gunpointmaleversusfemale', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'gunpointoldversusyoung', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'middlephalanxoutlineagegroup', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}

In [9]:

np.savetxt(f"./evaluation/inf_time_baseline_acc.txt", np.round(results.numpy(), 3), delimiter="\t", fmt='%.5f')

In [11]:
np.round(np.mean(results.numpy()), 3)

0.23

In [7]:
round(0.02357788, 3)

0.024

In [19]:
results = torch.zeros([15, 3])

for ds in range(15):
        args.DATASET = ds
        seed = args.SEED
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
        print(datainfo)

        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')
        # aug_model_exist = os.path.isfile(f'../AugLearnableFilters/models/{modelname}')
        # baseline_model_exist = os.path.isfile(f'../LastBaseline/LearnableFilters/models/{modelname}')
        
        # print(model_exist and aug_model_exist and baseline_model_exist)
        # print(model_exist and aug_model_exist)
        
        
        # if model_exist and aug_model_exist:
        if model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # aug_model = torch.load(f'../AugLearnableFilters/models/{modelname}', map_location=args.DEVICE)
            # baseline_model = torch.load(f'../LastBaseline/LearnableFilters/models/{modelname}', map_location=baseline_args.DEVICE)
            
            model.UpdateArgs(args)
            # aug_model.UpdateArgs(args)
            # baseline_model.UpdateArgs(baseline_args)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            state_dict = model.state_dict()
            num_theta = 0
            num_r = 0
            num_c = 0
            num_transistor = 0
            for name in state_dict:
                if "theta_" in name:
                    num_r += (state_dict[name] > 0).sum()
                elif ("rt_" in name) and ("InvRT" in name):
                    num_r += ((state_dict[name][0]>0) + (state_dict[name][2]>0) + (state_dict[name][4]>0))
                    num_transistor += 3 * (state_dict[name][5]>0 and state_dict[name][6]>0)
                elif ("rt_" in name) and ("TanhRT" in name):
                    num_r += ((state_dict[name][0]>0) + (state_dict[name][1]>0))
                    num_transistor += (state_dict[name][2]>0 and state_dict[name][3]>0) + (state_dict[name][4]>0 and state_dict[name][5]>0)
                elif "R_" in name:
                    num_r += (state_dict[name]>0).sum()
                elif "C_" in name:
                    num_c += (state_dict[name]>0).sum()
            
            results[ds][0] = num_theta
            results[ds][1] = num_r
            results[ds][2] = num_c

        else:
            results[ds] = float('nan')

{'dataname': 'cbf', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'distalphalanxtw', 'N_feature': 1, 'N_class': 6, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'freezerregulartrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'freezersmalltrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
{'dataname': 'gunpointagespan', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'gunpointmaleversusfemale', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'gunpointoldversusyoung', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 270, 'N_valid': 90, 'N_test': 91}
{'dataname': 'middlephalanxoutlineagegroup', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}

In [18]:
np.savetxt(f"./evaluation/count_device_baseline.txt", results.numpy().astype(int), delimiter="\t", fmt='%.5f')

In [10]:
results = torch.zeros([15,10,2])

for seed in range(10):
    for ds in range(15):
        args.SEED = seed
        args.DATASET = ds

        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')

        modelname = f"pLF_data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
        # print(modelname)
        
        if os.path.isfile(f'./models/{modelname}'):                
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            model.UpdateArgs(args)
            print(model.DEVICE)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            with torch.no_grad():
                state_dict = model.state_dict()
                accs_valid = []
                accs_test = []
                for _ in range(args.N_test):
                    for name in state_dict:
                        if name.endswith('rt_') or name.endswith('beta') or name.endswith('theta_'):
                            value = state_dict[name].to(args.DEVICE)
                            nosie = (((torch.rand(value.shape) * 2.) - 1.) * args.e_test + 1.).to(args.DEVICE)
                            state_dict[name] = value * nosie 
                            # print(name, mean.shape)
                    model.load_state_dict(state_dict)
        
                    accs_valid.append(evaluator(model, X_valid, y_valid))
                    accs_test.append(evaluator(model, X_test,  y_test))
                print(accs_valid, accs_test)
                results[ds,seed,0] = np.mean(accs_valid)
                results[ds,seed,1] = np.mean(accs_test)
        else:
            results[ds,seed,:] = float('nan')

cpu
[0.63, 0.725, 0.82, 0.71, 0.89, 0.68, 0.85, 0.79, 0.855, 0.765] [0.67, 0.645, 0.76, 0.755, 0.83, 0.895, 0.875, 0.805, 0.76, 0.74]
cpu
[0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47] [0.46, 0.46, 0.46, 0.46, 0.46, 0.46, 0.46, 0.46, 0.46, 0.46]
cpu
[0.765, 0.47, 0.425, 0.475, 0.405, 0.4, 0.515, 0.51, 0.415, 0.475] [0.8, 0.53, 0.445, 0.455, 0.495, 0.54, 0.47, 0.46, 0.445, 0.48]
cpu
[0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53] [0.54, 0.54, 0.54, 0.54, 0.54, 0.54, 0.54, 0.54, 0.54, 0.54]
cpu
[0.5833333333333334, 0.5444444444444444, 0.5444444444444444, 0.4666666666666667, 0.5611111111111111, 0.5777777777777777, 0.5777777777777777, 0.5777777777777777, 0.5611111111111111, 0.6166666666666667] [0.6153846153846154, 0.46703296703296704, 0.5494505494505495, 0.46153846153846156, 0.532967032967033, 0.5274725274725275, 0.5274725274725275, 0.5274725274725275, 0.5989010989010989, 0.532967032967033]
cpu
[0.7333333333333333, 0.7333333333333333, 0.7, 0.6944444444444444, 0.5

In [8]:
results

tensor([[[0.8820, 0.8830],
         [0.6955, 0.6745],
         [0.5985, 0.5940],
         [0.8200, 0.7880],
         [0.6530, 0.6310],
         [0.6190, 0.6020],
         [0.8430, 0.8560],
         [0.7210, 0.7125],
         [0.5330, 0.5390],
         [0.6340, 0.6535]],

        [[0.4700, 0.4600],
         [0.6040, 0.6415],
         [0.4700, 0.4600],
         [0.4700, 0.4600],
         [0.4700, 0.4600],
         [0.5935, 0.6145],
         [0.7090, 0.7500],
         [0.4700, 0.4600],
         [0.5120, 0.5565],
         [0.4700, 0.4600]],

        [[0.4810, 0.5335],
         [0.4970, 0.5760],
         [0.4930, 0.5195],
         [0.4955, 0.5010],
         [0.4995, 0.5000],
         [0.4940, 0.5180],
         [0.6195, 0.6430],
         [0.5275, 0.5145],
         [0.5015, 0.5100],
         [0.5550, 0.5825]],

        [[0.5045, 0.5045],
         [0.5300, 0.5400],
         [0.5630, 0.5735],
         [0.5300, 0.5400],
         [0.4820, 0.4780],
         [0.5300, 0.5400],
         [0.5300, 0.54

In [11]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,1], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation/1_var_test_top_{N_selected_seeds}_baseline_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

In [ ]:
print(datainfo)

In [1]:
results.shape

NameError: name 'results' is not defined

In [5]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,0], N_selected_seeds, dim=1, largest=True, sorted=True)
selected_results = results.gather(1, indices.unsqueeze(-1).expand(-1, -1, 2))
selected_test_mean = selected_results[:,:,-1].mean(1)
selected_test_std = selected_results[:,:,-1].std(1)
selected_test_results = torch.cat([selected_test_mean.unsqueeze(-1), selected_test_std.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation/top_{N_selected_seeds}_LPF_acc.txt", selected_test_results.numpy(), delimiter="\t", fmt='%.5f')


In [1]:
results

NameError: name 'results' is not defined